In [422]:
def readin(filename):
    with open(filename) as f:
        lines = f.readlines()

    #something to remove \n 
    str_lines = []
    for l in lines:
        if l[-1:]=="\n":
            str_lines.append(l[0:-1])
        else: 
            str_lines.append(l)

    #check all was fine
    # print(lines[-2:])
    # print(str_lines[-2:])
    return str_lines

elf_data = readin("Day16input.txt")

# --- Day 16: Proboscidea Volcanium ---

The sensors have led you to the origin of the distress signal: yet another handheld device, just like the one the Elves gave you. However, you don't see any Elves around; instead, the device is surrounded by elephants! They must have gotten lost in these tunnels, and one of the elephants apparently figured out how to turn on the distress signal.

The ground rumbles again, much stronger this time. What kind of cave is this, exactly? You scan the cave with your handheld device; it reports mostly igneous rock, some ash, pockets of pressurized gas, magma... this isn't just a cave, it's a volcano!

You need to get the elephants out of here, quickly. Your device estimates that you have 30 minutes before the volcano erupts, so you don't have time to go back out the way you came in.

You scan the cave for other options and discover a network of pipes and pressure-release valves. You aren't sure how such a system got into a volcano, but you don't have time to complain; your device produces a report (your puzzle input) of each valve's flow rate if it were opened (in pressure per minute) and the tunnels you could use to move between the valves.

There's even a valve in the room you and the elephants are currently standing in labeled AA. You estimate it will take you one minute to open a single valve and one minute to follow any tunnel from one valve to another. What is the most pressure you could release?

In [423]:
import networkx as nx
import operator as op
import collections as col
import math 
from functools import lru_cache

def mean(pq):
    flowrates = [fr[1] for fr in pq]
    return sum(flowrates)/len(flowrates)

def median(pq):
    flowrates = [fr[1] for fr in pq]
    flowrates.sort()
    midpoint = math.floor(len(flowrates)/2)
    return flowrates[midpoint]

def above_centile(pq, pc):
    flowrates = [fr[1] for fr in pq]
    flowrates.sort()
    point = math.floor(len(flowrates)*pc/100)
    return flowrates[point]

# (a, b) = V_test.gen_priorityQ_list_by_flowrate()
# print(average(a))
# print(average(b))
# print(a)
# print(b)
max_time = 30
start_point = "AA"

def view_graph(V):
    for n in nx.nodes(V):
        print(n, nx.get_node_attributes(V, "flowrate")[n], nx.edges(V, n))

#print(list(nx.get_node_attributes(V_test, "flowrate").items()))


For example, suppose you had the following scan output:

    

In [424]:
test_data = ["Valve AA has flow rate=0; tunnels lead to valves DD, II, BB",
"Valve BB has flow rate=13; tunnels lead to valves CC, AA",
"Valve CC has flow rate=2; tunnels lead to valves DD, BB",
"Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE",
"Valve EE has flow rate=3; tunnels lead to valves FF, DD",
"Valve FF has flow rate=0; tunnels lead to valves EE, GG",
"Valve GG has flow rate=0; tunnels lead to valves FF, HH",
"Valve HH has flow rate=22; tunnel leads to valve GG",
"Valve II has flow rate=0; tunnels lead to valves AA, JJ",
"Valve JJ has flow rate=21; tunnel leads to valve II"]



order: Ax, D20, Cx, B13, Ax, Ix, J21, Ix, Ax, D20, Ex, Fx, Gx, H22, Gx, Fx, E3, Dx, C2, 

In [425]:
class Elf_Volcano:

    def __init__(self, edges, labels):
        self.G = nx.Graph()
        #V.add_nodes_from(labels) # actually not needed, any unconnected nodes become unnecessary (unless start node...)
        self.G.add_edges_from(edges)
        nx.set_node_attributes(self.G, labels, "flowrate")
        (self.priorityQ, self.zeros) = self.gen_priorityQ_list_by_flowrate()
        self.accummulated_pressure = 0
        self.sum_open_pressure = 0
        self.current_valve = None
        self.target = None
        self.time_left = -1
        self.current_path = None
        self.open_valves = []
        # self.sp_all = nx.all_pairs_shortest_path_length(self.G)
        self.sp_all = nx.all_pairs_shortest_path(self.G)
        sp  = {}
        for n in self.G.nodes():
            sp[n] = nx.single_source_shortest_path_length(self.G, n)
        nx.set_node_attributes(self.G, sp, "sps")
        self.memo = []
        self.generated_sp_all = {}
        for n in self.sp_all:
            self.generated_sp_all[n[0]] = n[1]
        self.num_valves=len(self.priorityQ)-1 #number of non-zero valves left to open
        self.final_res = None
        

    @lru_cache()
    def get_flow(self, node):
        return nx.get_node_attributes(self.G, "flowrate")[node]
    
    @lru_cache()
    def get_sps(self, node):
        return nx.get_node_attributes(self.G, "sps")[node]

    @lru_cache()
    def get_node(self, node_name):
        return nx.nodes(self.G)[node_name]
    
    def gen_priorityQ_list_by_flowrate(self):
        v_list = [x for x in nx.get_node_attributes(self.G, "flowrate").items() if x[1] > 0]
        zeros = [x for x in nx.get_node_attributes(self.G, "flowrate").items() if x[1] == 0]
        v_list.sort(reverse=True, key = op.itemgetter(1))
        return v_list, zeros

    def adj(self, source, target):
        return self.get_sps(source)[target]

    def maximise_pressure(self, start_point, max_time):
        self.current_valve = start_point
        self.time_left = max_time
        self.open_valves = []
        # filler = []
        # for i in range(0, self.time_left+1):
        #     filler.append(".")
        # for n in range(0, num_valves+1):
        #     self.memo.append(filler)
        # #print(self.memo)
        self.start_tsp()
        print(self.target, self.current_path)
        
        while self.time_left>0:
            self.take_turn_tsp(max_time, self.num_valves)
        return self.accummulated_pressure

    def take_turn_tsp(self, max_time):
        if len(self.target)>0 and self.target[0] == self.current_valve:
            self.open_the_current_valve(max_time)
            self.move_to_tunnel(max_time)

    def time_tick(self, time_left, msg):
        self.time_left = self.time_left - 1
        self.accummulated_pressure = self.accummulated_pressure + self.sum_open_pressure
        print("== Minute "+ str(time_left-self.time_left)+  " == \n open valves: "+ str(self.open_valves)+" releasing "+str(self.sum_open_pressure)+ msg +"\n")

    def open_the_current_valve(self, time_left): 
        msg = "\nYou open valve "+ self.current_valve
        self.time_tick(time_left, msg)
        self.open_valves.append(self.current_valve)
        self.target.pop(0)
        if len(self.target)>0:
            self.current_path = self.generated_sp_all[self.current_valve][self.target[0]]
        else: 
            self.current_path = []
        #cvn = (self.current_valve, self.get_flow(self.current_valve))
        #print(self.target[0], self.current_path)
        #self.priorityQ.remove(cvn)
        self.sum_open_pressure = self.sum_open_pressure + self.get_flow(self.current_valve)
                
    def move_to_tunnel(self, time_left):
        if len(self.current_path)>0:
            for n in range(1, len(self.current_path)-1):
                if self.time_left>0:
                    self.current_valve = self.current_path[n+1] #we know there's an edge
                    msg = "\nYou move to "+ str(self.current_valve)
                    self.time_tick(time_left, msg)
        else: 
            msg = "\nYou move to "+ str(self.current_valve)
            for n in range(self.time_left, 0, -1):
                self.time_tick(time_left, msg)

    # adapting code from geeksforgeeks contributed by ng24_7 https://www.geeksforgeeks.org/traveling-salesman-problem-using-branch-and-bound-2/
    # Function to copy temporary solution
    # to the final solution
    def copyToFinal(self, curr_path):
        self.target[:self.num_valves + 1] = curr_path[:]
        self.target[self.num_valves] = curr_path[0]
        self.current_path = self.get_sps(self.current_valve, self.target[0])

    # Function to find the minimum edge cost
    # having an end at the vertex i
    def firstMin(self,i):
        min = self.time_left
        for v in range(0, self.num_valves):
            cost_i_v = self.adj(self.priorityQ[i][0], self.priorityQ[v][0])
            if cost_i_v < min and i != v:
                min = cost_i_v
        return min

    # function to find the second minimum edge
    # cost having an end at the vertex i
    def secondMin(self, i):
        first, second = self.time_left, self.time_left
        for v in range(0, self.num_valves):
            if i != v:
                cost_i_v = self.adj(self.priorityQ[i][0], self.priorityQ[v][0])
                if cost_i_v <= first:
                    second = first
                    first = cost_i_v
                elif(cost_i_v <= second and
                    cost_i_v != first):
                    second = cost_i_v
        return second

    # function that takes as arguments:
    # curr_bound -> lower bound of the root node
    # curr_weight-> stores the weight of the path so far
    # level-> current level while moving
    # in the search space tree
    # curr_path[] -> where the solution is being stored
    # which would later be copied to self.current_path[]
    def TSPRec(self, curr_bound, curr_weight,
                level, curr_path, visited):
        
        # base case is when we have reached level self.num_valves
        # which means we have covered all the nodes once
        if level == self.num_valves:
            
            # check if there is an edge from
            # last vertex in path back to the first vertex
            if self.adj(self.priorityQ[curr_path[level - 1]][0],self.priorityQ[curr_path[0]][0]) != 0:
                
                # curr_res has the total weight
                # of the solution we got
                curr_res = curr_weight + self.adj(self.priorityQ[curr_path[level - 1]][0], self.priorityQ[curr_path[0]][0])
                if curr_res < self.final_res:
                    self.copyToFinal(curr_path)
                    self.final_res = curr_res
            return
    
        # for any other level iterate for all vertices
        # to build the search space tree recursively
        for i in range(self.num_valves):
            
            # Consider next vertex if it is not same
            # (diagonal entry in adjacency matrix and
            #  not visited already)
            if (self.adj(self.priorityQ[curr_path[level-1]][0],self.priorityQ[i][0]) != 0 and
                                visited[i] == False):
                temp = curr_bound
                curr_weight += self.adj(self.priorityQ[curr_path[level - 1]][0], self.priorityQ[i][0])
    
                # different computation of curr_bound
                # for level 2 from the other levels
                if level == 1:
                    curr_bound -= ((self.firstMin(curr_path[level - 1]) +
                                    self.firstMin(i)) / 2)
                else:
                    curr_bound -= ((self.secondMin(curr_path[level - 1]) +
                                    self.firstMin(i)) / 2)
    
                # curr_bound + curr_weight is the actual lower bound
                # for the node that we have arrived on.
                # If current lower bound < self.final_res,
                # we need to explore the node further
                if curr_bound + curr_weight < self.final_res:
                    curr_path[level] = i
                    visited[i] = True
                    
                    # call TSPRec for the next level
                    self.TSPRec(curr_bound, curr_weight,
                        level + 1, curr_path, visited)
    
                # Else we have to prune the node by resetting
                # all changes to curr_weight and curr_bound
                curr_weight -= self.adj(self.priorityQ[curr_path[level - 1]][0],self.priorityQ[i][0])
                curr_bound = temp
    
                # Also reset the visited array
                visited = [False] * len(visited)
                for j in range(level):
                    if curr_path[j] != -1:
                        visited[curr_path[j]] = True

    def start_tsp(self):
        # Driver code
        # Adjacency matrix for the given graph
        # adj = [[0, 10, 15, 20],
        #     [10, 0, 35, 25],
        #     [15, 35, 0, 30],
        #     [20, 25, 30, 0]]
        #self.num_valves = 4
        
        # self.current_path[] stores the final solution
        # i.e. the // path of the salesman.
        self.current_path = [None] * (self.num_valves + 1)
        
        # visited[] keeps track of the already
        # visited nodes in a particular path
        visited = [False] * self.num_valves
        
        # Stores the final minimum weight
        # of shortest tour.
        self.final_res = self.time_left
        
        self.TSP(visited)
        
        print("Minimum cost :", self.final_res)
        print("Path Taken : ", end = ' ')
        for i in range(self.num_valves + 1):
            print(self.current_path[i], end = ' ')

    # This function sets up self.current_path
    def TSP(self, visited):
        # Calculate initial lower bound for the root node
        # using the formula 1/2 * (sum of first min +
        # second min) for all edges. Also initialize the
        # curr_path and visited array
        curr_bound = 0
        curr_path = [-1] * (self.num_valves + 1)
        visited = [False] * self.num_valves
    
        # Compute initial bound
        for i in range(self.num_valves):
            curr_bound += (self.firstMin(i) +
                        self.secondMin(i))
    
        # Rounding off the lower bound to an integer
        curr_bound = math.ceil(curr_bound / 2)
    
        # We start at vertex 1 so the first vertex
        # in curr_path[] is 0
        visited[0] = True
        curr_path[0] = 0
    
        # Call to TSPRec for curr_weight
        # equal to 0 and level 1
        self.TSPRec(curr_bound, 0, 1, curr_path, visited)
    

In [426]:
#    def take_turn_KS(self, max_time, num_valves):
#         if len(self.target)>0 and self.target[0] == self.current_valve:
#             self.open_the_current_valve(max_time)
#         self.move_to_tunnel(max_time)

        
#    # thank you CS Dojo on youtube for the base pseudocode for 0-1 knapsack (DP)

#     def start_KS(self, num_valves):
#         (result, next) = self.KS(num_valves, self.time_left)
#         print(result, next)
#         if len(next)>0:
#             self.target = next
#             self.current_path = self.generated_sp_all[self.current_valve][self.target[0]] #self.sp_all[self.current_valve][self.target] #self.current_path = 
#         return (result, next)


#     def KS(self, num_valves, time):
#         # if self.memo[num_valves][time] != ".":  #already memoed?
#         #     return self.memo[num_valves][time]
#         if num_valves==-1 or time==0:  #opened all valves? or run out of time?
#             result = 0
#             next_to_open = []
#         else: 
#             curr_valve_now = self.priorityQ[num_valves]#for debugging
#             cost = self.get_sps(self.current_valve)[self.priorityQ[num_valves][0]]
#             if cost > time or self.priorityQ[num_valves][0] in self.open_valves:   # costs too much to open it or it's already open
#                 (result, next_to_open) = self.KS(num_valves-1, time)   
#                 #result = self.KS(num_valves-1, time)   
#             else: 
#                 (tmp1r, tmp1n) = self.KS(num_valves-1, time) # ignore then look at options
#                 (tmp2r, tmp2n) = self.KS(num_valves-1, time-cost) #open and then lok at options
#                 tmp1r = tmp1r 
#                 tmp2r = tmp2r + self.priorityQ[num_valves][1] 
#                 if tmp1r > tmp2r:
#                     result = tmp1r
#                     next_to_open = tmp1n
#                 else:
#                     result = tmp2r
#                     next_to_open = [self.priorityQ[num_valves][0]]+tmp2n

#         # self.memo[num_valves][time] = (result, next_to_open)
#         return (result, next_to_open)

In [427]:
def parse_data(data):
    labels = {}
    edges = []
    for d in data:
        words = d.split()
        #identify edges
        for e in words[9:]:
            edges.append((words[1], e[0:2]))
        #identify flow rates for valves
        fr = words[4].split("=")
        fr[1] = fr[1].removesuffix(";")
        labels[words[1]] = eval(fr[1])
    V = Elf_Volcano(edges, labels)
    return V

In [428]:
# V_test = parse_data(test_data)
# # generated_sp_all = {}
# # for n in V_test.sp_all:
# #     generated_sp_all[n[0]] = n[1]
# #     #generated_sp_all[n.key()] = n.value()

# print(V_test.generated_sp_all[start_point]["II"])

In [429]:
V_test = parse_data(test_data)
# V_elf = parse_data(elf_data)
print(V_test.maximise_pressure(start_point, max_time))

TypeError: 'NoneType' object does not support item assignment

*brute force dynamic programming didn't work, let's try algorithms for knapsack problem solving and simplify this a bit*
*and use proper memoisation instead of cheating with caches*

In [ ]:
# # thank you CS Dojo on youtube for the base pseudocode for 0-1 knapsack (DP)

# memo = []
# time = V_test.time_left
# num_valves=len(V_test.priorityQ) #number of non-zero valves left to open
# filler = ''.ljust(time, ".") 

# for num_valves in range(0, 30):
#     memo[num_valves] = filler

# V_test = parse_data(test_data)

# #V_test.priorityQ:
# # wt = self.get_sps(source_valve)[target_valve]


# def KS(num_valves, time):
#     if memo[num_valves][time] != ".":  #already memoed?
#         return memo[num_valves][time]
#     elif num_valves==0 or time==0:  #opened all valves? or run out of time?
#         result = 0
#     else: 
#         cost = V_test.get_sps(V_test.current_valve)[V_test.priorityQ[num_valves][0]]
#         if cost > time:   # costs too much to open it
#             result = KS(num_valves-1, time)   
#         else: 
#             tmp1 = KS(num_valves-1, time)
#             tmp2 = V_test.priorityQ[num_valves][1] + KS(num_valves-1, time-cost)
#             result = max(tmp1, tmp2)
#     memo[num_valves][time] = result
#     return result

*old attempt from medium*

Suppose we have a knapsack which can hold int w = 10 weight units. We have a total of int n = 4 items to choose from, whose values are represented by an array int[] val = {10, 40, 30, 50} and weights represented by an array int[] wt = {5, 4, 6, 3}.

In [ ]:
# memo = []
# time = 30 
# num_valves=6 #number of non-zero valves left to open
# filler = ''.ljust(time, ".") 

# for num_valves in range(0, 30):
#     memo[num_valves] = filler

# flow_rates = {10, 40, 30, 50} 
# #priorityQ = list of nodes to iterate through
# #cost to reach node = shortest path from a given node. Will need a 2d array
# #weights represented by an array int[] wt = {5, 4, 6, 3}.
# #wt = self.get_sps(source_valve)[target_valve]

# # // Populate base cases
# # if num_valves == 0 then we can't open  anything else
# for r in range(0, time + 1):
#     memo[0][r] = 0;

# # if there is no time left, we can't open anything else
# for c in range(0, num_valves + 1):
#     memo[c][0] = 0;

# # // Main logic
# for valve in range(0, len(priorityQ)):
#     for t in range(1, time+1):#  (0 to time?)
#         maxValWithoutCurr = memo[valve - 1][t]; #// This is guaranteed to exist
#         maxValWithCurr = 0; #// We initialize this value to 0

#         weightOfCurr = self.get_sps(self.current_valve)[valve - 1]; #// We use item -1 to account for the extra row at the top
#         if (t >= weightOfCurr):  #// We check if the knapsack can fit the current item
#             maxValWithCurr = self.get_sps(self.current_valve)[valve - 1]; #// If so, maxValWithCurr is at least the value of the current item

#             int remainingCapacity = t - weightOfCurr; #// remainingCapacity must be at least 0
#             maxValWithCurr += memo[valve - 1][time-t]; #// Add the maximum value obtainable with the remaining capacity
#         memo[valve][t] = max(maxValWithoutCurr, maxValWithCurr); #// Pick the larger of the two

In [ ]:
# V_test_copy = parse_data(test_data)
# # pq = V_test_copy.priorityQ.copy()
# # pq.sort()
# # print(V_test_copy.set_targetDP(15, start_point, tuple(pq)))



All of the valves begin closed. You start at valve AA, but it must be damaged or jammed or something: its flow rate is 0, so there's no point in opening it. However, you could spend one minute moving to valve BB and another minute opening it; doing so would release pressure during the remaining 28 minutes at a flow rate of 13, a total eventual pressure release of 28 * 13 = 364. Then, you could spend your third minute moving to valve CC and your fourth minute opening it, providing an additional 26 minutes of eventual pressure release at a flow rate of 2, or 52 total pressure released by valve CC.



Making your way through the tunnels like this, you could probably open many or all of the valves by the time 30 minutes have elapsed. However, you need to release as much pressure as possible, so you'll need to be methodical. Instead, consider this approach:

    == Minute 1 ==
    No valves are open.
    You move to valve DD.

    == Minute 2 ==
    No valves are open.
    You open valve DD.

    == Minute 3 ==
    Valve DD is open, releasing 20 pressure.
    You move to valve CC.

    == Minute 4 ==
    Valve DD is open, releasing 20 pressure.
    You move to valve BB.

    == Minute 5 ==
    Valve DD is open, releasing 20 pressure.
    You open valve BB.

    == Minute 6 ==
    Valves BB and DD are open, releasing 33 pressure.
    You move to valve AA.

    == Minute 7 ==
    Valves BB and DD are open, releasing 33 pressure.
    You move to valve II.

    == Minute 8 ==
    Valves BB and DD are open, releasing 33 pressure.
    You move to valve JJ.

    == Minute 9 ==
    Valves BB and DD are open, releasing 33 pressure.
    You open valve JJ.

    == Minute 10 ==
    Valves BB, DD, and JJ are open, releasing 54 pressure.
    You move to valve II.

    == Minute 11 ==
    Valves BB, DD, and JJ are open, releasing 54 pressure.
    You move to valve AA.

    == Minute 12 ==
    Valves BB, DD, and JJ are open, releasing 54 pressure.
    You move to valve DD.

    == Minute 13 ==
    Valves BB, DD, and JJ are open, releasing 54 pressure.
    You move to valve EE.

    == Minute 14 ==
    Valves BB, DD, and JJ are open, releasing 54 pressure.
    You move to valve FF.

    == Minute 15 ==
    Valves BB, DD, and JJ are open, releasing 54 pressure.
    You move to valve GG.

    == Minute 16 ==
    Valves BB, DD, and JJ are open, releasing 54 pressure.
    You move to valve HH.

    == Minute 17 ==
    Valves BB, DD, and JJ are open, releasing 54 pressure.
    You open valve HH.

    == Minute 18 ==
    Valves BB, DD, HH, and JJ are open, releasing 76 pressure.
    You move to valve GG.

    == Minute 19 ==
    Valves BB, DD, HH, and JJ are open, releasing 76 pressure.
    You move to valve FF.

    == Minute 20 ==
    Valves BB, DD, HH, and JJ are open, releasing 76 pressure.
    You move to valve EE.

    == Minute 21 ==
    Valves BB, DD, HH, and JJ are open, releasing 76 pressure.
    You open valve EE.

    == Minute 22 ==
    Valves BB, DD, EE, HH, and JJ are open, releasing 79 pressure.
    You move to valve DD.

    == Minute 23 ==
    Valves BB, DD, EE, HH, and JJ are open, releasing 79 pressure.
    You move to valve CC.

    == Minute 24 ==
    Valves BB, DD, EE, HH, and JJ are open, releasing 79 pressure.
    You open valve CC.

    == Minute 25 ==
    Valves BB, CC, DD, EE, HH, and JJ are open, releasing 81 pressure.

    == Minute 26 ==
    Valves BB, CC, DD, EE, HH, and JJ are open, releasing 81 pressure.

    == Minute 27 ==
    Valves BB, CC, DD, EE, HH, and JJ are open, releasing 81 pressure.

    == Minute 28 ==
    Valves BB, CC, DD, EE, HH, and JJ are open, releasing 81 pressure.

    == Minute 29 ==
    Valves BB, CC, DD, EE, HH, and JJ are open, releasing 81 pressure.

    == Minute 30 ==
    Valves BB, CC, DD, EE, HH, and JJ are open, releasing 81 pressure.

This approach lets you release the most pressure possible in 30 minutes with this valve layout, 1651.

Work out the steps to release the most pressure in 30 minutes. What is the most pressure you can release?

*2454 is too high, 1885 and 1756 are too low*
*1981 is not right*

In [ ]:
# V_elf = parse_data(elf_data)
# print(start_point, max_time)
# print("RESULT = ",V_elf.maximise_pressure(start_point, max_time))